This is my first Release Code.

I learn from [this code](https://www.kaggle.com/josepc/rsna-effnet) so much,Thank you.

Please Run with My this code[[Create_TFRecord_for_RSNA-Radiogenomic](https://www.kaggle.com/hazigin/create-tfrecord-for-rsna-radiogenomic)]

In [ ]:
import os
import random
import sys
import numpy as np
from numpy.core.fromnumeric import _trace_dispatcher
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from pathlib import Path
import cv2
from sklearn.model_selection import train_test_split

In [ ]:
# limit the GPU memory growth
gpu = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpu))
if len(gpu) > 0:
    tf.config.experimental.set_memory_growth(gpu[0], True)

In [ ]:
height = 800
width = 800
channel = 8
batch_size 8
epochs = 30
seed = 26
input_depth = 4

In [ ]:
inputdatapath = "../input/MYTFRECORDSET"
outputpath = "./"

In [ ]:
class ClassificationModel(tf.keras.Model):
    def __init__(self,**kwargs):
        super(ClassificationModel, self).__init__(**kwargs)
        self.efficient_net = tf.keras.applications.resnet_v2.ResNet152V2(weights="imagenet",include_top=False,pooling='avg',input_tensor=None)
        self.Conv2D_filter = tf.keras.layers.Conv2D(3, kernel_size=3, padding='same',input_shape=(height,width, 4))
        self.dence64 = tf.keras.layers.Dense(64, activation='relu')
        self.dence1 = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, input_tensor, training=True):
        x = self.Conv2D_filter(input_tensor)
        x = self.efficient_net(x)
        x = self.dence64(x)
        return self.dence1(x) 

    def train_step(self, data):
        x ,y = data
        with tf.GradientTape() as tape:
            predictions = self(x, training=True)
            loss = self.compiled_loss(y,predictions,regularization_losses=self.losses)
            print(y,predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        self.compiled_metrics.update_state(y, predictions)
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        # Unpack the data
        x, y = data
        # Compute predictions
        y_pred = self(x, training=False)
        # Updates the metrics tracking the loss
        self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        # Update the metrics.
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}

In [ ]:
def deserialize_example(serialized_string):
    image_feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'MGMT_value': tf.io.FixedLenFeature([], tf.float32)
    }
    parsed_record = tf.io.parse_single_example(serialized_string, image_feature_description)
    image = tf.io.decode_raw(parsed_record['image'], tf.float64)
    image = tf.reshape(image,[height,width,4])
    
    label = parsed_record['MGMT_value']
    return image, label

In [ ]:
def argument_image(img,label):
    
    img = tf.cast(img, tf.float32) / 255.0

    splitted_img = tf.split(img, input_depth, axis=-1)

    augment_img = []
    for each_img in splitted_img:
        img = tf.repeat(each_img, repeats=3, axis=-1)
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_saturation(img, 0.9, 1.3)
        img = tf.image.random_contrast(img, 0.8, 1.2)
        img = tf.image.random_brightness(img, 0.2)
        img, _, _ = tf.split(img, 3, axis=-1)
        img = tf.expand_dims(img, 0)
        augment_img.append(img)
        
    img = tf.concat(augment_img, axis=-1)      
    img = tf.reshape(img, [height, width, input_depth])
    return img,label

def argument_image_val(img,label):   
    img = tf.cast(img, tf.float32) / 255.0
    return img,label

In [ ]:
model = ClassificationModel()
loss_func = tf.keras.losses.BinaryCrossentropy(from_logits=True)
opt = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(
optimizer=opt, 
loss=loss_func,
metrics=[tf.keras.metrics.AUC(), 
    tf.keras.metrics.BinaryCrossentropy(from_logits=True)]
)

In [ ]:
earlyStopping = tf.keras.callbacks.EarlyStopping(patience=3, min_delta=0.001, verbose=1,restore_best_weights=True)

In [ ]:
#Please rewirte path for your TFRecode Position
trainset = tf.data.TFRecordDataset(str(inputdatapath / "brain_train.tfrec"),compression_type="GZIP").map(deserialize_example).map(argument_image).batch(batch_size)
valnset = tf.data.TFRecordDataset(str(inputdatapath / "brain_val.tfrec"),compression_type="GZIP").map(deserialize_example).map(argument_image_val).batch(batch_size)

In [ ]:
history = model.fit(
trainset, 
epochs=25,
verbose=1,
callbacks=[earlyStopping],
validation_data=valnset)

In [ ]:
tf.keras.backend.clear_session()

train_score = model.evaluate(trainset, batch_size=batch_size)
y_train = model.predict(trainset)
validation_score = model.evaluate(valnset, batch_size=batch_size)
y_val = model.predict(valnset)
print('Train loss :', train_score[0])
print('Train accuracy :', train_score[1])
print('validation loss :', validation_score[0])
print('validation accuracy :', validation_score[1])

In [ ]:
weightdatapath = Path(outputpath,'weight')
model.save_weights(Path(weightdatapath,"weight.ckpt"))

If you feel this code is usefull

Please up Vote!!

Thank You!!